In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint


In [3]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.games import print_model as pm
from SBMLLint.games.som import SOM

In [21]:
num = 50
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Kinetic_modelling_of_Amadori_degradation "Martins2003_AmadoriDegradation">
v1: DFG -> E1;
v2: DFG -> E2;
v3: DFG -> Gly + Cn;
v4: E1 -> Gly + _3DG;
v5: _3DG -> Cn;
v6: _3DG -> FA;
v7: E2 -> Gly + _1DG;
v8: _1DG -> Cn;
v9: _1DG -> AA;
v10: E1 -> Gly + Man;
v11: E1 -> Gly + Glu;
v12: Man -> Glu;
v13: Glu -> _3DG;
v14: Gly + Cn -> Mel;
v15: Cn -> AA + FA + MG;
v16: E2 -> Gly + Fru;


In [5]:
simple = SimpleSBML(model)

In [6]:
print(simple._getReactions())
len(simple._getReactions())

[<Reaction R1>, <Reaction R2>, <Reaction R3>, <Reaction R4>, <Reaction R5>, <Reaction R6>, <Reaction R7>, <Reaction R8>, <Reaction R9>, <Reaction R10>, <Reaction R11>, <Reaction R12>, <Reaction R13>, <Reaction R14>]


14

In [7]:
Reaction.reactions = []
Molecule.molecules = []
Reaction.initialize(simple)
for reaction in Reaction.reactions:
    print(reaction.category)

reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_n_n
reaction_1_1
reaction_1_1
reaction_1_1
reaction_n_n
reaction_1_1
reaction_n_1
reaction_1_1


In [8]:
#Reaction.reactions

In [9]:
print(Molecule.molecules)

[ADP, NAD, halfglucose, ATP, NADH, pyruvate, lactate, CoA, AcCoA, PO4, AcP, Ac, AcO, EtOH, AcLac, AcetoinIn, AcetoinOut, Butanediol, O2]


In [10]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [11]:
class Arc(object):
    
    def __init__(self, source, destination, reaction):
        """
        :param SOM source:
        :param SOM destination:
        :param Reaction reaction:
        """
        self.source = source
        self.destination = destination
        self.reaction = reaction

In [12]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [13]:
#Reaction.reactions[1].reactants[0].molecule

#     @classmethod
#     def findGraph(cls, molecule):
#         """
#         :param Molecule molecule:
#         :return MESGraph/False:
#         """
#         som = SOM.findSOM(molecule)
#         for graph in cls.all:
#             for node in graph.nodes:
#                 if node == som:
#                     return graph
#         return False
        

In [14]:
#ARROW = "->"

class MESGraph(nx.DiGraph):
    """
    The MESGraph class represents a collection of SOMs as nodes
    and their inequality relationships as edges (arcs). 
    Mass inequality between SOMs from reactions can help us
    detect their relationship.
    Type I Error occurs when we find inequality between two molecules
    in the same SOM, because each element in a SOM has the same weight.
    Type II Error implies there is cyclism between molecules, such as
    A < B < C < ... < A, which is physically impossible. 
    """
    # all = []

    def __init__(self, soms):
        """
        :param list-SOM soms:
        """
        super(MESGraph, self).__init__()
        self.add_nodes_from(soms)
        self.identifier = self.makeId()

    def __repr__(self):
        return self.identifier
    
    def makeId(self):
        identifier = ""
        if self.edges:
            for edge in self.edges:
                identifier = identifier + str(edge[0]) + cn.ARC_ARROW + str(edge[1]) + "\n"

        for key, node in enumerate(nx.isolates(self)):
            identifier = identifier + str(node)
            if key < (len(list(nx.isolates(self)))-1):
                identifier = identifier + cn.KINETICS_SEPARATOR
                
        return identifier
    
    def processUniUniReaction(self, reaction):
        """
        Process a 1-1 reaction to update nodes.
        Uses SOM.merge(reaction) method. 
        :param Reaction reactions:
        """
        if reaction.category != cn.REACTION_1_1:
            pass
        else:
            reactant_som = SOM.findSOM(reaction.reactants[0].molecule)
            product_som = SOM.findSOM(reaction.products[0].molecule)
            if reactant_som == product_som:
                pass
            else:
                new_som = SOM.merge(reaction)
                self.remove_node(reactant_som)
                self.remove_node(product_som)
                self.add_node(new_som)
                return new_som
    
    def processUniMultiReaction(self, reaction):
        """
        Process a 1-n reaction to add arcs.
        Since the mass of reactant is greater than
        that of each product, it adds arcs by
        addArc(source=products, destination=reactant). 
        :param Reaction reaction:
        """
        if (reaction.category != cn.REACTION_1_n):
            pass
        else:
            destination = [reaction.reactants[0].molecule]
            source = [product.molecule for product in reaction.products]
            self.addArc(source, destination, reaction)

    def processMultiUniReaction(self, reaction):
        """
        Process a n-1 reaction to add arcs.
        Since the mass of product is greater than
        that of each reactant, it adds arcs by
        addArc(source=reactants, destination=product). 
        :param Reaction reaction:
        """
        if (reaction.category != cn.REACTION_n_1):
            pass
        else:
            destination = [reaction.products[0].molecule]
            source = [reactant.molecule for reactant in reaction.reactants]
            self.addArc(source, destination, reaction)
    
    def addArc(self, source, destination, reaction):
        """
        Check Type I Error and Add arcs 
        using two list of molecules (source/destination).
        :param list-Molecule source:
        :param list-Molecule destination:
        """
        arcs = itertools.product(source, destination)
        for arc in arcs:
            if not self.checkTypeOneError(arc, reaction):
                self.add_edge(SOM.findSOM(arc[0]), SOM.findSOM(arc[1]), reaction=reaction)
            else:
                continue
    
    def checkTypeOneError(self, arc, inequality_reaction=None):
        """
        Check Type I Error of an arc.
        If both source and destination are found
        in the same SOM, send error message and return True.
        If not, return False.
        :param tuple-Molecule arc:
        :param Reaction inequality_reaction:
        :return bool:
        """
        som1 = SOM.findSOM(arc[0])
        som2 = SOM.findSOM(arc[1])
        if som1 == som2:
            print("We have Type I Error...")
            print(arc[0], " and ", arc[1], " have the same weight by")
            for equality_reaction in list(som1.reactions):
                print(equality_reaction)
            ##
            print("\nHowever, reaction \"", inequality_reaction, 
                  "\" implies ", arc[0], " < ", arc[1])
            print()
            return True
        else:
            return False
    
    def analyze(self, reactions):
        """
        Sort list of reactions and process them.
        Add arcs or sending error messages using
        checkTypeOneError or checkTypeTwoError.
        :param list-Reaction reactions:
        """
        uniuni = []
        unimulti = []
        multiuni = []
        multimulti = []
        for reaction in reactions:
            if reaction.category == cn.REACTION_1_1:
                uniuni.append(reaction)
            elif reaction.category == cn.REACTION_1_n:
                unimulti.append(reaction)
            elif reaction.category == cn.REACTION_n_1:
                multiuni.append(reaction)
            elif reaction.category == cn.REACTION_n_n:
                multimulti.append(reaction)
            
        for reaction in uniuni:
            self.processUniUniReaction(reaction)
        for reaction in unimulti:
            self.processUniMultiReaction(reaction)
        for reaction in multiuni:
            self.processMultiUniReaction(reaction)
            
        self.identifier = self.makeId()
        return self


In [15]:
m = MESGraph(SOM.soms)
m.nodes

NodeView(({ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}))

In [16]:
m

{ADP};{NAD};{halfglucose};{ATP};{NADH};{pyruvate};{lactate};{CoA};{AcCoA};{PO4};{AcP};{Ac};{AcO};{EtOH};{AcLac};{AcetoinIn};{AcetoinOut};{Butanediol};{O2}

In [17]:
m.edges

OutEdgeView([])

In [18]:
m.analyze(Reaction.reactions)
m.nodes

NodeView(({NAD}, {halfglucose}, {NADH}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {Butanediol}, {O2}, {AcLac, AcetoinIn, AcetoinOut, pyruvate}, {ADP, ATP}))

In [19]:
m.edges

OutEdgeView([({NADH}, {NAD}), ({O2}, {NAD})])

In [20]:
m

{NADH}->{NAD}
{O2}->{NAD}
{halfglucose};{lactate};{CoA};{AcCoA};{PO4};{AcP};{Ac};{AcO};{EtOH};{Butanediol};{AcLac, AcetoinIn, AcetoinOut, pyruvate};{ADP, ATP}

In [453]:
m.edges

OutEdgeView([({Gly}, {DFG, E1, E2}), ({Gly}, {Mel}), ({MG}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}), ({Fru}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {DFG, E1, E2}), ({AA, Cn, FA, Glu, Man, _1DG, _3DG}, {Mel})])

In [456]:
list(nx.isolates(m))

[]

In [424]:
identifier = ""
if m.edges:
    for edge in m.edges:
        identifier = identifier + str(edge[0]) + "->" + str(edge[1]) + "\n"
    
for key, node in enumerate(nx.isolates(m)):
    identifier = identifier + str(node)
    if key < (len(list(nx.isolates(m)))-1):
        identifier = identifier + cn.KINETICS_SEPARATOR
print(identifier)

{Gly}->{DFG, E1, E2}
{Gly}->{Mel}
{MG}->{AA, Cn, FA, Glu, Man, _1DG, _3DG}
{Fru}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{DFG, E1, E2}
{AA, Cn, FA, Glu, Man, _1DG, _3DG}->{Mel}



{DFG};{E1};{E2};{Gly};{Cn};{_3DG};{FA};{_1DG};{AA};{Man};{Glu};{Mel};{MG};{Fru}

In [323]:
str(edge[0])

'{Gly}'

In [343]:
nodes = list(m.nodes)
print(nodes)

[{Gly}, {Mel}, {MG}, {Fru}, {DFG, E1, E2}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}]


In [373]:
for x in nx.isolates(m):
    print(x)
    print(type(x))

{ATP, Ade, DNA, GTP, Gua, HX, IMP, PRPP, R5P, RNA, SAM, SAMP, UA, XMP, Xa, dATP, dGTP}
<class 'SBMLLint.games.som.SOM'>


In [341]:
nx.ancestors(m, list(m.edges)[0][1])

{{Fru}, {MG}, {Gly}, {AA, Cn, FA, Glu, Man, _1DG, _3DG}}

In [339]:
list(m.edges)[0][0]

{Gly}

In [374]:
m

;{ATP}+{HX}+{Ade}+{dATP}+{DNA}+{IMP}+{PRPP}+{RNA}+{SAMP}+{dGTP}+{Gua}+{GTP}+{XMP}+{Xa}+{SAM}+{R5P}+{UA}+

In [376]:
nx.isolates(m)

<generator object isolates.<locals>.<genexpr> at 0x1521ab2c50>

In [22]:
a = {1}.union({2})

In [23]:
a

{1, 2}

In [24]:
a.add(3)

In [25]:
a

{1, 2, 3}

In [26]:
SOM.soms

[{E2},
 {Gly},
 {Cn},
 {_3DG},
 {FA},
 {_1DG},
 {AA},
 {Man},
 {Glu},
 {Mel},
 {MG},
 {Fru}]